In [3]:
import sys
sys.path.insert(0, 'C:/Users/user/Documents/iplab/stock_factor_lab')

In [4]:
from get_data import Data
import backtest
from combinations import sim_conditions
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data=Data()

In [6]:
'''
下載資料
1. price:股價相關
2. report:財報相關
'''
data.get("price:close")

company_symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2000-01-04,7.5067,5.3558,5.5019,2.6105,5.4752,5.4740,6.2886,6.5692,6.4118,1.7259,...,NaN,1.5950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,7.7020,5.7189,5.5304,2.6507,5.6920,5.5840,6.2573,7.0125,6.4685,1.7978,...,NaN,1.6852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,7.3766,5.7734,5.7584,2.7042,5.6920,5.7159,6.4450,7.4961,6.6387,1.8377,...,NaN,1.7605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-07,7.3549,5.7734,5.8155,2.6908,5.6920,5.7379,6.4450,8.0200,6.6955,1.8457,...,NaN,1.7605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-10,7.5935,5.8097,5.9580,2.6908,5.6107,5.6719,6.3512,8.5439,6.8941,1.8537,...,NaN,1.7605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-25,34.3500,40.3000,18.8000,29.3500,15.7000,18.1500,18.7500,19.0000,53.2000,57.7000,...,20.15,36.2500,20.15,17.60,22.8,75.3,24.20,168.0,27.30,18.45
2023-12-26,34.5000,40.5500,18.8000,29.4000,15.7500,18.1000,18.5500,19.0500,53.5000,57.4000,...,20.20,36.4000,20.15,18.05,22.6,76.8,24.20,167.0,27.25,18.30
2023-12-27,34.6500,40.9000,18.9000,29.5500,15.8000,18.1000,18.7500,19.0500,54.7000,57.3000,...,20.15,36.8000,20.40,17.90,22.4,75.8,24.45,167.0,27.10,18.20


# 策略回測

In [5]:
'''
示範單因子選股
'''

# 這裡表示根據roe由大到小排序
# divide_slice: 切割成幾群，以及如何排序
roe_q1 = data.get("report:roe").divide_slice(10, ascending=False)['Quantile_1']
roe_q1

company_symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
2000-05-15,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2000-08-31,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2000-11-15,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2001-03-31,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2001-05-15,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-15,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2023-08-31,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2023-11-15,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False
2024-03-31,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [6]:
# 代表每一期roe前10%的股票數量
roe_q1.sum(axis=1)

2000-05-15      0
2000-08-31      0
2000-11-15      0
2001-03-31      0
2001-05-15      0
             ... 
2023-05-15    182
2023-08-31    182
2023-11-15    182
2024-03-31    182
2024-05-15    182
Length: 97, dtype: int64

In [7]:
'''
投資策略: 買入每一期roe前10%的股票，每個月換一次股票
'''
report = backtest.sim(roe_q1, resample="M", data=data)
report.display()

,annualized_rate_of_return,sharpe,max_drawdown,win_ratio
,14.18%,27.62,-44.93%,41.24%


,entry_date,exit_date,entry_sig_date,exit_sig_date,position,period,entry_index,exit_index,return,mae,gmfe,bmfe,mdd,pdays,weight,next_weights
stock_id,,,,,,,,,,,,,,,,
1215,2023-09-01,2024-06-03,2023-08-31,2024-05-31,0.005495,181,5860,6041,0.121716,-0.037315,0.212664,0.044593,-0.092106,144.0,0.000000,0.000000
1231,2024-04-01,2024-06-03,2024-03-29,2024-05-31,0.005495,42,5999,6041,0.003259,-0.033401,0.022267,0.000000,-0.033401,11.0,0.000000,0.000000
1232,2024-04-01,2024-06-03,2024-03-29,2024-05-31,0.005495,42,5999,6041,-0.019502,-0.020478,0.000000,0.000000,-0.020478,0.0,0.000000,0.000000
1436,2024-04-01,2024-06-03,2024-03-29,2024-05-31,0.005495,42,5999,6041,0.346829,-0.031250,0.464286,0.000000,-0.201220,33.0,0.000000,0.000000
1457,2024-04-01,2024-06-03,2024-03-29,2024-05-31,0.005495,42,5999,6041,0.093160,-0.014851,0.262376,0.017327,-0.154902,35.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910322,2019-09-02,NaT,2019-08-30,NaT,0.005650,1174,4886,-1,0.343068,-0.047392,0.767279,0.066352,-0.459368,1144.0,0.005495,0.005495
912000,2024-04-01,NaT,2024-03-29,NaT,0.005495,61,5999,-1,0.765609,-0.034441,0.779062,0.328784,-0.273351,45.0,0.005495,0.005495
9911,2024-04-01,NaT,2024-03-29,NaT,0.005495,61,5999,-1,0.113637,0.000000,0.162545,0.000000,-0.102646,60.0,0.005495,0.005495
